In [ ]:
%load_ext autoreload
%autoreload 2
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
from GraPL import hyperparameter_profiles, segment_bsds
import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def run_experiment(base_params, lambdas, num_trials=2):
    results_base_dir = f'experiment_results/lambda_ablation'
    aggregate_scores = {}
    for lambda_ in lambdas:
        print(f'Running experiment with lambda_={lambda_}...')
        results_dir = f'{results_base_dir}/{lambda_}'
        trials = []
        mean_over_trials = {}
        with tqdm.tqdm(total=num_trials * 200) as progress_bar:
            for trial_num in range(num_trials):
                params = base_params.copy()
                params['lambda_'] = lambda_
                params['seed'] = trial_num
                trial_results_dir = f'{results_dir}/{trial_num}'
                trial_scores = segment_bsds(results_dir=trial_results_dir, resume=True, progress_bar=progress_bar, **params)
                image_ids = list(trial_scores.keys())
                metrics = trial_scores[image_ids[0]].keys()
                trial_scores = {metric: np.mean([trial_scores[id][metric] for id in image_ids]) for metric in metrics}
                trials.append(trial_scores)
        for metric in trials[0]:
            mean_over_trials[metric] = np.mean([trial[metric] for trial in trials])
        aggregate_scores[lambda_] = mean_over_trials
        print(f'Scores for lambda_={lambda_}: {mean_over_trials["accuracy"]}')
    return aggregate_scores

base_params = hyperparameter_profiles["best_miou"]

lambdas = [0, 1, 2, 4, 8, 16, 32, 64, 128, 256]

aggregate_scores = run_experiment(base_params, lambdas, num_trials=10)